In [61]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


True

In [62]:
from src.inference import load_batch_of_features_from_store, get_future_predictions, load_model_from_registry, get_model_predictions
from src.data import transform_ts_data_into_features_target

ts_prices = load_batch_of_features_from_store()  # get historical feature-data from feature-store
ts_prices


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,sma_5,sma_20,ema_5,ema_20,vol_5,daily_return,price_diff,volume_sma_5,volume_change
341,60362338.0,165.8939,166.37,164.87,167.81,164.20,540017.0,2022-08-02 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,6.036234e+07,0.000000
691,60362338.0,165.8939,166.37,164.87,167.81,164.20,540017.0,2022-08-03 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,6.036234e+07,0.000000
363,60362338.0,165.8939,166.37,164.87,167.81,164.20,540017.0,2022-08-04 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,6.036234e+07,0.000000
698,60362338.0,165.8939,166.37,164.87,167.81,164.20,540017.0,2022-08-05 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,6.036234e+07,0.000000
352,60362338.0,165.8939,166.37,164.87,167.81,164.20,540017.0,2022-08-06 00:00:00+00:00,165.754,169.627000,164.870000,164.870000,1.948854,0.000000,0.00,6.286661e+07,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,69545387.0,206.7390,205.30,207.23,209.99,201.07,921946.0,2024-08-06 04:00:00+00:00,213.098,218.944500,211.836310,217.883106,5.131917,-0.009748,-2.04,1.027162e+08,-0.417457
737,119382508.0,207.8010,199.09,209.27,213.50,196.00,1661493.0,2024-08-07 00:00:00+00:00,211.388,218.301000,211.434207,217.238593,3.157328,0.000000,0.00,1.160331e+08,0.000000
741,60109650.0,210.7375,206.90,209.82,213.64,206.39,741607.0,2024-08-07 04:00:00+00:00,211.090,218.226500,211.164207,217.115192,3.542831,0.012498,2.59,9.421107e+07,-0.135677
739,60109650.0,210.7375,206.90,209.82,213.64,206.39,741607.0,2024-08-08 00:00:00+00:00,209.788,217.502000,210.716138,216.420411,1.992855,0.000000,0.00,8.458946e+07,0.000000


In [63]:
model = load_model_from_registry()  

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


In [66]:
# GET HISTORICAL DATA PREDICTIONS
n_previous_days = 12
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size) # convert ts-data from feature-store into features/targets for training

historical_predictions = get_model_predictions(model, features, ts_prices) 
historical_predictions = historical_predictions.reset_index(drop=False)  # Creates a new 'index' column
historical_predictions.rename(columns={'index': 'id'}, inplace=True)  # get a primary key for it 
historical_predictions

Number of sequences/examples: 728
Last val: 172.82666666666665
Target val: 173.19
FEATURES
(728, 13, 16)


,id,predicted_prices
0,0,172.928325
1,1,173.088198
2,2,173.990845
3,3,174.434349
4,4,171.685435
...,...,...
723,723,209.323291
724,724,209.893192
725,725,208.682686
726,726,211.269024


In [67]:
# GET FUTURE DATA PREDICTIONS
num_days = 10
future_predictions = get_future_predictions(num_days, ts_prices, model)
# future_predictions['datetime'] = pd.to_datetime(future_predictions['datetime'])
future_predictions

,future_predicted_prices,datetime
0,138.243773,2024-08-10
1,209.249745,2024-08-11
2,211.179950,2024-08-12
3,212.257026,2024-08-13
4,212.473013,2024-08-14
5,211.613065,2024-08-15
6,212.358109,2024-08-16
7,212.211253,2024-08-17
8,212.395906,2024-08-18
9,213.077101,2024-08-19


In [69]:
# SAVE HISTORICAL PREDICTIONS 
from src.inference import get_feature_store
import src.config as config

# connect to the feature group, keep in mind these are the names and metadata from this feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name="model_predictions_historical2",
    version=1,
    description="Predictions generated by model on historical data",
    primary_key = ['id'],
    event_time='id',
)

feature_group.insert(historical_predictions, write_options={"wait_for_job": False})


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/835736/fs/831559/fg/1083729


Uploading Dataframe: 0.00% |          | Rows 0/728 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_historical2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/model_predictions_historical2_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x162a9d670>, None)

In [72]:
# SAVE FUTURE PREDICTIONS 
from src.inference import get_feature_store
import src.config as config

# connect to the feature group, keep in mind these are the names and metadata from this feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name="model_predictions_future2",
    version=1,
    description="Predictions generated by model on future data",
    primary_key = ["datetime"],
    event_time='datetime',
)

feature_group.insert(future_predictions, write_options={"wait_for_job": False})


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/835736/fs/831559/fg/1083730


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_future2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/model_predictions_future2_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x162c339b0>, None)

In [28]:
# # LOAD HISTORICAL PREDICTIONS FROM STORE TO CHECK
# feature_store = get_feature_store()

# # Retrieve the feature group
# feature_group = feature_store.get_feature_group(
#     name="model_prediction_historical",
#     version=1
# )

# # Read the data from the feature group
# historical_predictions_df = feature_group.read()
# historical_predictions_df

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


,predicted_prices,datetime
0,172.928325,2024-01-30 05:00:00+00:00
1,173.088198,2023-01-03 05:00:00+00:00
2,173.990845,2023-04-12 04:00:00+00:00
3,174.434349,2022-08-22 04:00:00+00:00
4,171.685435,2022-11-23 05:00:00+00:00
...,...,...
723,209.893192,2024-05-09 04:00:00+00:00
724,208.682686,2024-06-21 04:00:00+00:00
725,211.269024,2023-03-30 04:00:00+00:00
726,210.146128,2022-10-17 04:00:00+00:00


In [30]:
# LOAD FUTURE PREDICTIONS FROM STORE TO CHECK
feature_store = get_feature_store()

# Retrieve the feature group
feature_group = feature_store.get_feature_group(
    name="model_prediction_future",
    version=1
)

# Read the data from the feature group
future_predictions_df = feature_group.read()
future_predictions_df  # make sure to sort

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


,future_predicted_prices,datetime
0,138.243773,2024-08-10 00:00:00+00:00
1,212.473013,2024-08-14 00:00:00+00:00
2,212.257026,2024-08-13 00:00:00+00:00
3,211.613065,2024-08-15 00:00:00+00:00
4,211.179950,2024-08-12 00:00:00+00:00
5,213.077101,2024-08-19 00:00:00+00:00
6,212.358109,2024-08-16 00:00:00+00:00
7,212.395906,2024-08-18 00:00:00+00:00
8,209.249745,2024-08-11 00:00:00+00:00
9,212.211253,2024-08-17 00:00:00+00:00
